In [ ]:
%load_ext rpy2.ipython

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import scipy.signal as ss
import scipy.io

import sys
sys.path.insert(1, '../modules/')
from cleanedfunctions import *
plt.style.use('default')
import statsmodels.api as sm
import os

In [ ]:
### Utils
cd "dfa in different bands"
cd "newdata"

final = [5.0, 6.0, 9.0, 11.0, 12.0, 14.0, 16.0, 17.0, 18.0, 19.0, 22.0, 23.0, 27.0, 30.0, 33.0, 35.0, 37.0, 40.0, 41.0, 45.0, 46.0, 47.0, 49.0, 52.0, 53.0, 56.0, 58.0, 62.0, 64.0, 66.0, 67.0, 68.0, 71.0]

def mapping(sub, final):
    return final.index(sub) + 1

cleans = []
bands = [[4,8],[8,13],[14,30],[31,60]]
xminn = np.log2( (3/np.array(bands)[:,0])*500)
xminn[-1] = xminn[-1] + 2

In [ ]:
### Load data
for band in range(4):
    o = 0
    clean = pd.DataFrame(columns = ['Silence', 'DFAexp', 'sub_id', 'ch' ],  index = np.arange(0,588,1), dtype = 'float')
    for file in os.listdir():
        dfa_sil_params = np.load(file)[band]
        ntrial = dfa_sil_params.shape[0]
        dfa_sil_params = np.array(dfa_sil_params).reshape(ntrial,3,-1)
        dfa_sil_subjects = [[] for r in range(ntrial)]
        for r in range(ntrial):
            c = plot_fluct(dfa_sil_params[r,0],dfa_sil_params[r,1],dfa_sil_params[r,2],xmin = xminn[band], xmax = 13.4)
            clean.iloc[o]["DFAexp"] = float(c[0])
            if "." in file[4:6]:
                f = file[4]
            else:
                f = file[4:6]
            clean.iloc[o]["sub_id"]  = mapping(int(f), final)
            clean.iloc[o]["ch"] = int(r)
            o+=1
    clean["Silence"] = ["1" for r in range(283) ]+ ["2" for r in range(305)]
    cleans.append(clean)  

# Analysis - whole dataset

In [ ]:
band = 0

new = pd.DataFrame(cleans[band])

df = new
nsub = 33
df1 = (df[df["Silence"] == "1"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))


df1 = (df[df["Silence"] == "2"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))

In [ ]:
%%R -i new

library(lmerTest)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data = new,REML = FALSE)
print(summary(m))

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

In [ ]:
### code to do it in python:
import statsmodels.formula.api as smf
md = smf.mixedlm("DFAexp ~ Silence", new, groups=new["sub_id"], re_formula="~Silence")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
prv = cleans[1]

In [ ]:
%%R -i prv
## all random
library(lmerTest)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data = prv, REML = FALSE)

print(summary(m)$coefficients[1,1])
print(summary(m)$coefficients[1,2])
print(summary(m)$coefficients[2,1])
print(summary(m)$coefficients[2,5])

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

In [ ]:
band = 3

new = pd.DataFrame(cleans[band])

df = new
df1 = (df[df["Silence"] == "1"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))


df1 = (df[df["Silence"] == "2"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))

In [ ]:
%%R -i new
## all random
library(lmerTest)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data = new,REML = FALSE)
print(summary(m)$coefficients[1,1])
print(summary(m)$coefficients[1,2])
print(summary(m)$coefficients[2,1])
print(summary(m)$coefficients[2,5])


In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

## Different order of the languages

In [ ]:
### ORDER OF THE LANGUAGES
order = pd.read_excel('../../../../../Downloads/ordine lingue.xlsx')#1 en, 2 fr, 3 sp

In [ ]:
subjects = order[order["last language"]==3]["subject"].values
subjects

In [ ]:
subject_french =[]
subjects = order[order["last language"]==2]["subject"].values

for ll in range(len(subjects)):
    if subjects[ll] in final:
        subject_french.append(mapping(int(subjects[ll]), final))

subject_en =[]
subjects = order[order["last language"]==1]["subject"].values

for ll in range(len(subjects)):
    if subjects[ll] in final:
        subject_en.append(mapping(int(subjects[ll]), final))
        
subjects = order[order["last language"]==3]["subject"].values
subject_span = []
for ll in range(len(subjects)):
    if subjects[ll] in final:
        subject_span.append(mapping(int(subjects[ll]), final))

In [ ]:
np.array(list(map(len, [subject_french, subject_en, subject_span]))).sum()

In [ ]:
i = 0
for r in new["sub_id"]:
    if r in subject_french:
        i+= 1
print(i)

In [ ]:
cleans = []
bands = [[4,8],[8,13],[14,30],[31,60]]
xminn = np.log2( (3/np.array(bands)[:,0])*500)
xminn[-1] = xminn[-1] + 2
for band in range(4):
    o = 0
    clean = pd.DataFrame(columns = ['Silence', 'DFAexp', 'sub_id', 'lingua' ],  index = np.arange(0,588,1),dtype = float)
    for file in os.listdir():
        dfa_sil_params = np.load(file)[band]
        ntrial = dfa_sil_params.shape[0]
        dfa_sil_params = np.array(dfa_sil_params).reshape(ntrial,3,-1)
        dfa_sil_subjects = [[] for r in range(ntrial)]
        for r in range(ntrial):
            c = plot_fluct(dfa_sil_params[r,0],dfa_sil_params[r,1],dfa_sil_params[r,2],xmin = xminn[band], xmax = 13.4)
            clean.iloc[o]["DFAexp"] = float(c[0])
            if "." in file[4:6]:
                f = file[4]
            else:
                f = file[4:6]
            if mapping(int(f), final) in subject_french:
                clean.iloc[o]["lingua"] = 1
            elif mapping(int(f), final) in subject_en:
                clean.iloc[o]["lingua"] = 2
            elif mapping(int(f), final) in subject_span:
                clean.iloc[o]["lingua"] = 3

            clean.iloc[o]["sub_id"]  = mapping(int(f), final)
            clean.iloc[o]["ch"] = int(r)
            o+=1
    clean["Silence"] = ["1" for r in range(283) ]+ ["2" for r in range(305)]
    cleans.append(clean) 

In [ ]:
new = cleans[0]

In [ ]:
df = new
nsub = 33
df1 = (df[df["Silence"] == "1"][["DFAexp", "sub_id", "lingua"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))


df1 = (df[df["Silence"] == "2"][["DFAexp", "sub_id", "lingua"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id) + lingua + lingua*Silence + (1 + lingua*Silence|sub_id) + (1 + lingua|sub_id)', data=new, )
print(summary(m))

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = FALSE)
print(summary(m))
print(coef(summary(m))[1,1])
print(coef(summary(m))[1,2])
print(coef(summary(m))["Silence2","Estimate"])
print(coef(summary(m))["Silence2","Pr(>|t|)"])

In [ ]:
frenchs = []
for g in range(len(cleans)):
    french = pd.DataFrame(columns = ["Silence", "DFAexp", "sub_id", "ch"],index = np.arange(0,221,1), dtype = 'float')
    new = cleans[g]
    i = 0
    for r in range(len(subject_french)):
        v = new.loc[new["sub_id"] == int(subject_french[r])]
        for k in range(len(v["sub_id"].values)):
            french.iloc[i] = v.iloc[k]
            i+= 1
    
    ## The lines below are just to rename the subject ID. not necessary
    dd = french["sub_id"].values.copy()
    o = 0
    for l in dd:
        french["sub_id"].iloc[o] = mapping(l,list(set(dd)))
        o+=1
    ##
    frenchs.append(french)

In [ ]:
engs = []

for g in range(len(cleans)):

    eng = pd.DataFrame(columns = ["Silence", "DFAexp", "sub_id", "ch"],index = np.arange(0,203,1), dtype = 'float')
    new = cleans[g]
    
    i = 0
    for r in range(len(subject_en)):
        v = new.loc[new["sub_id"] == int(subject_en[r])]
        for k in range(len(v["sub_id"].values)):
            eng.iloc[i] = v.iloc[k]
            i+= 1
    ##
    dd = eng["sub_id"].values.copy()
    o = 0
    for l in dd:
        eng["sub_id"].iloc[o] = mapping(l,list(set(dd)))
        o+=1
    ##
    engs.append(eng)

In [ ]:
spans = []
for g in range(len(cleans)):
    span = pd.DataFrame(columns = ["Silence", "DFAexp", "sub_id", "ch"],index = np.arange(0,164,1), dtype = 'float')
    new = cleans[g]
    i = 0
    for r in range(len(subject_span)):
        v = new.loc[new["sub_id"] == int(subject_span[r])]
        for k in range(len(v["sub_id"].values)):
            span.iloc[i] = v.iloc[k]
            i+= 1
    ##
    o = 0
    dd = span["sub_id"].values.copy()
    for l in dd:
        span["sub_id"].iloc[o] = mapping(l,list(set(dd)))
        
        o+=1
    ##
    spans.append(span)

## Theta band

In [ ]:
new = frenchs[0]

In [ ]:
df = new
nsub = 33
df1 = (df[df["Silence"] == "1"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))


df1 = (df[df["Silence"] == "2"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = FALSE)
print(summary(m))
print(coef(summary(m))[1,1])
print(coef(summary(m))[1,2])
print(coef(summary(m))["Silence2","Estimate"])
print(coef(summary(m))["Silence2","Pr(>|t|)"])

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

In [ ]:
new = spans[0]

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = FALSE)
print(summary(m))
print(coef(summary(m))[1,1])
print(coef(summary(m))[1,2])
print(coef(summary(m))["Silence2","Estimate"])
print(coef(summary(m))["Silence2","Pr(>|t|)"])

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

In [ ]:
new = engs[0]

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = FALSE)
print(summary(m))
print(coef(summary(m))[1,1])
print(coef(summary(m))[1,2])
print(coef(summary(m))["Silence2","Estimate"])
print(coef(summary(m))["Silence2","Pr(>|t|)"])

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

## Gamma Band

In [ ]:
band = 3
new = frenchs[band]

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = FALSE)
print(summary(m))
print(coef(summary(m))[1,1])
print(coef(summary(m))[1,2])
print(coef(summary(m))["Silence2","Estimate"])
print(coef(summary(m))["Silence2","Pr(>|t|)"])

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

In [ ]:
new = spans[band]

In [ ]:
df = new
nsub = 33
df1 = (df[df["Silence"] == "1"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))


df1 = (df[df["Silence"] == "2"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = FALSE)
print(summary(m))
print(coef(summary(m))[1,1])
print(coef(summary(m))[1,2])
print(coef(summary(m))["Silence2","Estimate"])
print(coef(summary(m))["Silence2","Pr(>|t|)"])

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))

In [ ]:
new = engs[band]

In [ ]:
df = new
nsub = 33
df1 = (df[df["Silence"] == "1"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))


df1 = (df[df["Silence"] == "2"][["DFAexp", "sub_id", "ch"]])
mean1 = np.mean(df1["DFAexp"])
np.std(df1["DFAexp"])
print(mean1, np.std(df1["DFAexp"]))

In [ ]:
%%R -i new

library(lmerTest)
library(lme4)
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = FALSE)
print(summary(m))
print(coef(summary(m))[1,1])
print(coef(summary(m))[1,2])
print(coef(summary(m))["Silence2","Estimate"])
print(coef(summary(m))["Silence2","Pr(>|t|)"])

In [ ]:
%%R -i new
## all random
 
library(lmerTest)
val <- FALSE
m<-lmer('DFAexp ~ Silence + (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m))

#library(lme4)
m2<-lmer('DFAexp ~ (1 + Silence|sub_id)', data=new, REML = val)
print(AIC(m2))
print(anova(m2,m))